In [1]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import os
from PIL import Image
from tqdm import tqdm
import numpy as np
from datasets import load_dataset
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions

In [3]:
class FastFoodClassifier:
    def __init__(self, input_shape=(224, 224, 3), num_classes=10):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = self.build_model()

    def build_model(self):
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=self.input_shape)

        for layer in base_model.layers:
            layer.trainable = False

        model = models.Sequential()
        model.add(base_model)
        model.add(layers.Flatten())
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(self.num_classes, activation='softmax'))

        return model

    def load_data(self, dataset_name="andrewsunanda/fast_food_image_classification"):
        dataset = load_dataset(dataset_name, split="train")

        data = dataset['train']

        return data

    def train_and_evaluate(self, train_generator, test_generator, num_epochs=10, weights_filepath='fast_food_classifier_weights.h5'):
        checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', save_best_only=True, save_weights_only=True, mode='min', verbose=1)

        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        history = self.model.fit(train_generator, epochs=num_epochs, validation_data=test_generator, callbacks=[checkpoint])

        test_loss, test_acc = self.model.evaluate(test_generator)
        print(f'Test accuracy: {test_acc}')

    def save_model_weights(self, filepath='fast_food_classifier_weights.h5'):
        self.model.save_weights(filepath)

    def predict_image(self, image_path):
        img = image.load_img(image_path, target_size=self.input_shape)
        img_array = image.img_to_array(img)
        img_array = preprocess_input(img_array.reshape(1, *img_array.shape))
        
        labels = {
            0: 'Baked Potato',
            1: 'Burger',
            2: 'Crispy Chicken',
            3: 'Donut',
            4: 'Fries',
            5: 'Hot Dog',
            6: 'Pizza',
            7: 'Sandwich',
            8: 'Taco',
            9: 'Taquito'
        }
        prediction = self.model.predict(img_array)
        prediction = labels[np.argmax(prediction)]
        return prediction

In [4]:
classifier = FastFoodClassifier()

train_dir = 'data/train'
test_dir = 'data/train'
batch_size = 32
num_epochs = 10

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=classifier.input_shape[:2], batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=classifier.input_shape[:2], batch_size=batch_size, class_mode='categorical')

# Train and evaluate the model
classifier.train_and_evaluate(train_generator, test_generator, num_epochs=num_epochs)

Found 2997 images belonging to 10 classes.
Found 2997 images belonging to 10 classes.
Epoch 1/10
94/94 [==============================] - ETA: 0s - loss: 6.3923 - accuracy: 0.3570
Epoch 1: val_loss improved from inf to 1.29156, saving model to fast_food_classifier_weights.h5
94/94 [==============================] - 439s 5s/step - loss: 6.3923 - accuracy: 0.3570 - val_loss: 1.2916 - val_accuracy: 0.5883
Epoch 2/10
94/94 [==============================] - ETA: 0s - loss: 1.5870 - accuracy: 0.4731
Epoch 2: val_loss improved from 1.29156 to 1.00872, saving model to fast_food_classifier_weights.h5
94/94 [==============================] - 531s 6s/step - loss: 1.5870 - accuracy: 0.4731 - val_loss: 1.0087 - val_accuracy: 0.7027
Epoch 3/10
94/94 [==============================] - ETA: 0s - loss: 1.4992 - accuracy: 0.4868
Epoch 3: val_loss did not improve from 1.00872
94/94 [==============================] - 624s 7s/step - loss: 1.4992 - accuracy: 0.4868 - val_loss: 1.1009 - val_accuracy: 0.6703

In [8]:
classifier.predict_image('pizza.jpeg')

1/1 [==============================] - 0s 96ms/step


'Pizza'

In [9]:
classifier.predict_image('burger.jpeg')

1/1 [==============================] - 0s 100ms/step


'Burger'

In [10]:
classifier.predict_image('baked-potato.jpeg')

1/1 [==============================] - 0s 112ms/step


'Baked Potato'